In [2]:
import pandas as pd
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) # 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
code_df = code_df[['회사명', '종목코드']] # 한글로된 컬럼명을 영어로 바꿔준다. 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})

code_df.head()

,name,code
0,CJ,001040
1,CJ씨푸드,011150
2,HSD엔진,082740
3,KG케미칼,001390
4,OCI,010060


In [64]:
import requests, time
from bs4 import BeautifulSoup
import progressbar

# 종목 이름을 입력하면 종목에 해당하는 코드를 불러와 
# 네이버 금융(http://finance.naver.com)에 넣어줌 
def get_url(item_name, code_df): 
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False) 
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code) 
    print("요청 URL = {}".format(url)) 
    
    r = requests.get(url)
    source = BeautifulSoup(r.content, 'html.parser')
    max_page = source.find_all('table', align='center')
    max_page = max_page[0].find_all('td', class_='pgRR')
    max_page = max_page[0].a.get('href')
    
    return url, int(max_page[max_page.rfind('page=')+5:])

# 신라젠의 일자데이터 url 가져오기 
item_name='신라젠' 
url, max_page = get_url(item_name, code_df) 
# 일자 데이터를 담을 df라는 DataFrame 정의 
df = pd.DataFrame() # 1페이지에서 20페이지의 데이터만 가져오기 

for page in progressbar.progressbar(range(1, max_page+1)):
    try:
        pg_url = '{url}&page={page}'.format(url=url, page=page) 
        df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True) 
    except:
        print(item_name, page)
        break
        
    if page % 50 == 0:
        time.sleep(1.50)

# df.dropna()를 이용해 결측값 있는 행 제거 
df = df.dropna() # 상위 5개 데이터 확인하기 
df.head()

# 한글로 된 컬럼명을 영어로 바꿔줌 
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['date'], ascending=True)


요청 URL = http://finance.naver.com/item/sise_day.nhn?code=215600


100% (54 of 54) |########################| Elapsed Time: 0:00:13 Time:  0:00:13


In [56]:
df

,date,close,diff,open,high,low,volume
798,2016-12-06,12850,650,13500,15050,12400,13725568
797,2016-12-07,11800,1050,12750,13150,11750,4291145
796,2016-12-08,11200,600,11850,12300,10900,4501845
793,2016-12-09,11900,700,11350,12300,11250,3958017
792,2016-12-12,11550,350,12150,12300,11450,1711610
791,2016-12-13,13300,1750,11550,13800,11400,6621075
790,2016-12-14,12950,350,13050,13700,12850,3493116
789,2016-12-15,13600,650,12800,14350,12700,5434936
785,2016-12-16,13750,150,13750,14150,13400,1815132
784,2016-12-19,13600,150,13800,14250,13500,1981956


In [65]:
# 필요한 모듈 import 하기 
import plotly.offline as offline 
import plotly.graph_objs as go 

# jupyter notebook 에서 출력 
offline.init_notebook_mode(connected=True) 
trace = go.Scatter( x=df.date, y=df.close, name=item_name) 
data = [trace] 
# data = [celltrion] 
layout = dict(
                title='{}의 종가(close) Time Series'.format(item_name), 
                xaxis=dict( 
                    rangeselector=dict( 
                        buttons=list([ 
                            dict(count=1, label='1m', step='month', stepmode='backward'), 
                            dict(count=3, label='3m', step='month', stepmode='backward'), 
                            dict(count=6, label='6m', step='month', stepmode='backward'), 
                            dict(count=12, label='12m', step='month', stepmode='backward'), 
                            dict(count=36, label='36m', step='month', stepmode='backward'), 
                            dict(step='all') ]) ), 
                    rangeslider=dict(), 
                    type='date' 
                )
        )

fig = go.Figure(data=data, layout=layout) 
offline.iplot(fig)

In [47]:

url = get_url('삼성전자', code_df)


요청 URL = http://finance.naver.com/item/sise_day.nhn?code=005930


570